In [1]:
import pandas as pd
import mlflow
import warnings
from sklearn.linear_model import ElasticNet
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../data/wine-quality.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [3]:
remote_server_uri = "http://0.0.0.0:5000"
mlflow.set_tracking_uri(remote_server_uri)
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [4]:
exp_name = "mlflow-demo"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='/home/balamurugan/mlflow-walkthrough/mlruns/380943166029257618', creation_time=1712316677885, experiment_id='380943166029257618', last_update_time=1712316677885, lifecycle_stage='active', name='mlflow-demo', tags={}>

In [5]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)
    train, test = train_test_split(data)
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y

In [6]:
def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    data_path = "../data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [7]:
train()

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: /home/balamurugan/mlflow-walkthrough/mlruns/380943166029257618/14ca4fa468f1445f9971cb566f3f194a/artifacts


In [8]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.search_experiments()
print(experiments)

[<Experiment: artifact_location='/home/balamurugan/mlflow-walkthrough/mlruns/380943166029257618', creation_time=1712316677885, experiment_id='380943166029257618', last_update_time=1712316677885, lifecycle_stage='active', name='mlflow-demo', tags={}>, <Experiment: artifact_location='/home/balamurugan/mlflow-walkthrough/mlruns/0', creation_time=1712316664801, experiment_id='0', last_update_time=1712316664801, lifecycle_stage='active', name='Default', tags={}>]


In [9]:
_run = client.get_run(run_id="291f1cc2c7b749f389b2292deca3d7d4")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6278761410160691, 'r2': 0.12678721972772689, 'rmse': 0.82224284975954}, params={'alpha': '0.5', 'l1_ratio': '0.5'}, tags={'deployed': '05-04-2024 (17:55:25.230140)',
 'mlflow.log-model.history': '[{"run_id": "291f1cc2c7b749f389b2292deca3d7d4", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-04-05 11:42:43.762791", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.2", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.0", '
                             '"serialization_format": "cloudpickle",

In [10]:
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)

In [11]:
_run = client.get_run(run_id="291f1cc2c7b749f389b2292deca3d7d4")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6278761410160691, 'r2': 0.12678721972772689, 'rmse': 0.82224284975954}, params={'alpha': '0.5', 'l1_ratio': '0.5'}, tags={'deployed': '05-04-2024 (18:14:23.537180)',
 'mlflow.log-model.history': '[{"run_id": "291f1cc2c7b749f389b2292deca3d7d4", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-04-05 11:42:43.762791", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.2", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.3.0", '
                             '"serialization_format": "cloudpickle",